### Imports

In [ ]:
import sys

sys.path.append("../..")

from pathlib import Path
import pydicom
import os
import numpy as np
import matplotlib.pyplot as plt
import dicom_image_tools as dit

from dicom_image_tools.roi.square_roi import SquareRoi
from dicom_image_tools.helpers.voxel_data import (
    VoxelData,
)
from dicom_image_tools.helpers.point import (
    Point,
)

from utils.plot_utils import (
    _remove_ax_ticks,
    _create_colorbar,
)

from utils.dit_utils import (
    _fetch_roi_trace,
    _find_images_lims,
    _get_image_center,
    _calculate_image_lims_exclude_cornerbox,
    _get_first_key,
)

In [ ]:
path_date1 = Path(r'C:\Users\maxh01\git\case_trasig_xios\72012143_leveranskontroll')
path_date2 = Path(r'C:\Users\maxh01\git\case_trasig_xios\72012143_uppföljningskontroll')

data_date1 = dit.import_dicom_from_folder(path_date1)
data_date2 = dit.import_dicom_from_folder(path_date2)

datas = [data_date1, data_date2]

for data in datas:
    for study in data:
        for serie in data[study].Series:
            serie.import_image()
            serie.sort_images_on_acquisition_time()

data = dict(
    date1 = dict(),
    date2 = dict()
)

def _fetch_series_info(serie):
    res = dict()
    
    res["kV"] = serie.kV
    res["ms"] = serie.ms
    res["date"] = [idx.AcquisitionDate for idx in serie.CompleteMetadata]
    res["ID"] = [idx.DetectorID for idx in serie.CompleteMetadata]
    res["mat"] = serie.ImageVolume
    res["VoxelData"] = serie.VoxelData
    res["dose"] = [idx.EntranceDoseInmGy for idx in serie.CompleteMetadata]
    
    

    return res

im_idx = 4

for study in data_date1:
    for serie in data_date1[study].Series:
        tmp = _fetch_series_info(serie)
        kV = int(tmp['kV'][0])
        kV_tag = f'{kV}kv'
        data["date1"][kV_tag] = tmp

for study in data_date2:
    for serie in data_date2[study].Series:
        tmp = _fetch_series_info(serie)
        kV = int(tmp['kV'][0])
        kV_tag = f'{kV}kv'
        data["date2"][kV_tag] = tmp

In [ ]:
row_date1, row_date2, row_diff = 0, 1, 2

col_60, col_70 = 0, 1
lims_60, lims_70 = [1200-150, 1200+150], [1950-150, 1950+150] 
lims_diff = [-500, +500]
im_idx = 4
cmap = plt.cm.gray.reversed()

fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(7, 5))

# 60kV, date 1
date, kv, lims, row, col = "date1", "60kv", lims_60, row_date1, col_60
ax = axs[row, col]
mat = data[date][kv]["mat"][im_idx]
im = ax.imshow(mat, vmin=lims[0], vmax=lims[1], cmap=cmap, interpolation=None)
ax.set(ylabel=data[date][kv]["date"][im_idx])
ax.set(title=f'{data[date][kv]["kV"][im_idx]} kV, {data[date][kv]["ms"][im_idx]} ms')
_remove_ax_ticks(ax)

# 60kV, date 2
date, kv, lims, row, col = "date2", "60kv", lims_60, row_date2, col_60
ax = axs[row, col]
mat = data[date][kv]["mat"][im_idx]
im = ax.imshow(mat, vmin=lims[0], vmax=lims[1], cmap=cmap, interpolation=None)
ax.set(ylabel=data[date][kv]["date"][im_idx])
_remove_ax_ticks(ax)
_create_colorbar(fig, im, ax, height="200%", width="7.5%", loc="lower right", bbox_to_anchor=(0.1,0,1,1))

# 60kV, diff
ax = axs[row_diff, col]
mat = data["date2"][kv]["mat"][im_idx] - data["date1"][kv]["mat"][im_idx]
im = ax.imshow(mat, vmin=lims_diff[0], vmax=lims_diff[1], cmap=plt.cm.RdBu_r, interpolation=None)
ylabel = f'{data[date][kv]["date"][im_idx]}-{data[date][kv]["date"][im_idx]}'
ax.set(ylabel='residual')
_remove_ax_ticks(ax)
_create_colorbar(fig, im, ax, height="85%", width="7.5%", loc="lower right")

# 70kV, date 1
date, kv, lims, row, col = "date1", "70kv", lims_70, row_date1, col_70
ax = axs[row, col]
mat = data[date][kv]["mat"][im_idx]
im = ax.imshow(mat, vmin=lims[0], vmax=lims[1], cmap=cmap, interpolation=None)
ax.set(title=f'{data[date][kv]["kV"][im_idx]} kV, {data[date][kv]["ms"][im_idx]} ms')
_remove_ax_ticks(ax)

# 70kV, date 2
date, kv, lims, row, col = "date2", "70kv", lims_70, row_date2, col_70
ax = axs[row, col]
mat = data[date][kv]["mat"][im_idx]
im = ax.imshow(mat, vmin=lims[0], vmax=lims[1], cmap=cmap, interpolation=None)
_remove_ax_ticks(ax)
_create_colorbar(fig, im, ax, height="200%", width="7.5%", loc="lower right", bbox_to_anchor=(0.1,0,1,1))

# 70kV, diff
ax = axs[row_diff, col]
mat = data["date2"][kv]["mat"][im_idx] - data["date1"][kv]["mat"][im_idx]
im = ax.imshow(mat, vmin=lims_diff[0], vmax=lims_diff[1], cmap=plt.cm.RdBu_r, interpolation=None)
ax.set(ylabel='residual')
_remove_ax_ticks(ax)
_create_colorbar(fig, im, ax, height="85%", width="7.5%", loc="lower right")


title1 = data["date1"]["60kv"]["ID"][im_idx]
title2 = data["date2"]["60kv"]["ID"][im_idx]
plt.suptitle(f"sensor: {title1} / {title2}")
plt.subplots_adjust(wspace=0, hspace=0)    
plt.savefig("flat_field_comparison.pdf", bbox_inches='tight', dpi=2000)
#plt.savefig("comp.pdf", bbox_inches='tight')
plt.show()


In [ ]:
rows, cols=dict(), dict()
rows["60kv"], rows["70kv"] = 0, 1

fig, ax = plt.subplots()

for date_instance in ["date1", "date2"]:
    for kv in ["60kv", "70kv"]:
        roi = SquareRoi(
            height=100,
            width=100,
            center=_get_image_center(image=data[date_instance][kv]["mat"][0]),
            pixel_size=data[date_instance][kv]["VoxelData"][0],
            roi_size_in_pixels=True,
        )
        label = f'{data[date_instance][kv]["date"][0]} ({kv})'
        im = ax.plot(
            data[date_instance][kv]["ms"],
            [roi.get_mean(image=mat) for mat in data[date_instance][kv]["mat"]],
            '--*', label=label)
        ax.set(xticks=data[date_instance]["60kv"]["ms"], ylim=[0, 2**12])
        plt.xticks(rotation=90, ha='right')
        ax.legend()
        ax.set(xlabel='exposure time', ylabel='mean pixel value (central)')

fig.set_size_inches(10,6)
plt.savefig("pixel_resonse_comparison.pdf", bbox_inches='tight')
plt.show()



In [ ]:
import PIL.Image
cmap = plt.cm.gray

path_ex1 = Path(r'C:\Users\maxh01\git\case_trasig_xios\ex1.png')
path_ex2 = Path(r'C:\Users\maxh01\git\case_trasig_xios\ex2.png')
im1 = np.array(PIL.Image.open(path_ex1))[:,:,0]
im2 = np.array(PIL.Image.open(path_ex2))[:,:,0]

fig, axs = plt.subplots(ncols=2)

ax = axs[0]
mat = im1
im = ax.imshow(mat, cmap=cmap)
ax.set(title='clinical example 1')
ax.set(ylabel=f'ID: {title1}')
_remove_ax_ticks(ax)

ax = axs[1]
mat = np.rot90(np.rot90(im2))
im = ax.imshow(mat, cmap=cmap)
ax.set(title='clinical example 2')
ax.set(ylabel=f'ID: {title1}')
_remove_ax_ticks(ax)

plt.savefig("clin.pdf")

plt.savefig("clinical_examples.pdf", bbox_inches='tight', dpi=2000)


In [ ]:
date

In [ ]:
data[date]["70kv"]